# Homework week 1: IMDB list

Nhóm mình chỉ có mình chị đi buổi present đợt trước nên c tóm tắt lại 2 ý quan trọng của Kai trong buổi đó nhé: 
1. Bài present phải có 1 câu hỏi chủ đề. Nghĩa là mình đang đi tìm câu trả lời cho câu hỏi nào, problem gì. 
2. Bài present phải có structure mạch lạc & logic. 

Với dataset IMDB này, câu hỏi của bt khá rõ ràng: "In this project, we take IMDB SCORE as response variable and focus on operating predictions by analyzing the rest of variables in the IMDB 5000 movie data. The results can help film companies to understand the secret of generating a COMMERCIAL SUCCESS movie".  
Tuy nhiên chị có 2 remarks ở đây: 
1. "a commercial success movie" : thành công về mặt thương mại nghĩa là tỉ suất lợi nhuận cao so với vốn bỏ ra (doanh thu trừ đi chi phí). Vậy phải focus vào 2 cột budget & gross để tính ra lợi nhuận thương mại. 
2. "IMDB score = response variable": ok dựa vào Imdb để quan sát các yếu tố khác thì cũng được thôi, Imdb là một biến khá bias, tuy nhiên dùng nó để quan sát thì cũng tạm được so với các biến khác trong dataset. 

Theo mạch này, chị đề xuất chủ đề của bài nhóm mình sẽ là "What are the common points of top commercial success movies?" nghĩa là mình sẽ đứng ở góc nhìn của 1 nhà đầu tư, đi tìm điểm chung của các bộ phim thành công về mặt thương mại, từ đó rút ra advise cho investor.

Structure của bài: 
1. Exploring dataset: get to know the dataframe, get a few first observation. 

2. Commercial success metrics: c đề nghị dùng ROI để đo độ success của each movie. Bước này sẽ bổ sung cột ROI của từng movie. 

3. Processing data: gồm 2 phần là Cleaning data (bỏ các row n/a, null, dummies) và Processing dataP (tách các genres ra để observe ở bước sau như Hưng đề xuất, sort ROI descending order, lấy list top_100_success etc...). Tại sao bh mới clean data? vì mình quan sát ROI thôi chứ các yếu tố khác ko quá quan trọng, thiếu 1 2 row cũng đc chị suppose. 

4. Observe data: plot dataframe top_100_success có được ở bước 3 theo nhiều angle khác nhau để soi xem có những điểm chung lớn nào. 

5. Extra: observe top_100_failure, top_100_imdb_score để xem có j đối lập hoàn toàn không?? nếu nó cũng pattern y chang nhóm top_100_success thì chết bà... 

6. Conclusion

## 1. EXPLORING DATASET

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('movie_metadata.csv')

In [4]:
data.shape

(5043, 28)

In [36]:
data.head(3)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000


In [6]:
data.describe(include='all')

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
count,5024,4939,4993.000000,5028.000000,4939.000000,5020.000000,5030,5036.000000,4.159000e+03,5043,...,5022.000000,5031,5038,4740,4.551000e+03,4935.000000,5030.000000,5043.000000,4714.000000,5043.000000
unique,2,2398,NaN,NaN,NaN,NaN,3032,NaN,NaN,914,...,NaN,47,65,18,NaN,NaN,NaN,NaN,NaN,NaN
top,Color,Steven Spielberg,NaN,NaN,NaN,NaN,Morgan Freeman,NaN,NaN,Drama,...,NaN,English,USA,R,NaN,NaN,NaN,NaN,NaN,NaN
freq,4815,26,NaN,NaN,NaN,NaN,20,NaN,NaN,236,...,NaN,4704,3807,2118,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,140.194272,107.201074,686.509212,645.009761,NaN,6560.047061,4.846841e+07,NaN,...,272.770808,NaN,NaN,NaN,3.975262e+07,2002.470517,1651.754473,6.442138,2.220403,7525.964505
std,NaN,NaN,121.601675,25.197441,2813.328607,1665.041728,NaN,15020.759120,6.845299e+07,NaN,...,377.982886,NaN,NaN,NaN,2.061149e+08,12.474599,4042.438863,1.125116,1.385113,19320.445110
min,NaN,NaN,1.000000,7.000000,0.000000,0.000000,NaN,0.000000,1.620000e+02,NaN,...,1.000000,NaN,NaN,NaN,2.180000e+02,1916.000000,0.000000,1.600000,1.180000,0.000000
25%,NaN,NaN,50.000000,93.000000,7.000000,133.000000,NaN,614.000000,5.340988e+06,NaN,...,65.000000,NaN,NaN,NaN,6.000000e+06,1999.000000,281.000000,5.800000,1.850000,0.000000
50%,NaN,NaN,110.000000,103.000000,49.000000,371.500000,NaN,988.000000,2.551750e+07,NaN,...,156.000000,NaN,NaN,NaN,2.000000e+07,2005.000000,595.000000,6.600000,2.350000,166.000000
75%,NaN,NaN,195.000000,118.000000,194.500000,636.000000,NaN,11000.000000,6.230944e+07,NaN,...,326.000000,NaN,NaN,NaN,4.500000e+07,2011.000000,918.000000,7.200000,2.350000,3000.000000


In [7]:
data.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
m

Observation: 
- nhiều cột type là float64 c cảm thấy ko cần thiết. Tại toàn số lớn triệu triệu ngàn ngàn ko mà để số lẻ thập phân chi ko biết --> round up bọn nó cho thành interger hết được ko hẻ mấy em??

In [28]:
data.count()

color                        5024
director_name                4939
num_critic_for_reviews       4993
duration                     5028
director_facebook_likes      4939
actor_3_facebook_likes       5020
actor_2_name                 5030
actor_1_facebook_likes       5036
gross                        4159
genres                       5043
actor_1_name                 5036
movie_title                  5043
num_voted_users              5043
cast_total_facebook_likes    5043
actor_3_name                 5020
facenumber_in_poster         5030
plot_keywords                4890
movie_imdb_link              5043
num_user_for_reviews         5022
language                     5031
country                      5038
content_rating               4740
budget                       4551
title_year                   4935
actor_2_facebook_likes       5030
imdb_score                   5043
aspect_ratio                 4714
movie_facebook_likes         5043
dtype: int64

Observation: 
- nhìn vào 2 cột gross và budget chỉ có 4159 & 4551 row filled/5043 rows ta biết bảng data này sẽ bị drop khá nhiều đây. Nhưng còn tầm 4000 row cho datafram thì cũng rất ổn rồi.
- why? vì các nhà sản xuất thường giữ kín thông tin chính xác về chi phí & doanh thu vì nó liên quan tới tỉ lệ chia chác của đoàn làm phim nhoé. 

In [29]:
data.nunique()

color                           2
director_name                2398
num_critic_for_reviews        528
duration                      191
director_facebook_likes       435
actor_3_facebook_likes        906
actor_2_name                 3032
actor_1_facebook_likes        878
gross                        4035
genres                        914
actor_1_name                 2097
movie_title                  4917
num_voted_users              4826
cast_total_facebook_likes    3978
actor_3_name                 3521
facenumber_in_poster           19
plot_keywords                4760
movie_imdb_link              4919
num_user_for_reviews          954
language                       47
country                        65
content_rating                 18
budget                        439
title_year                     91
actor_2_facebook_likes        917
imdb_score                     78
aspect_ratio                   22
movie_facebook_likes          876
dtype: int64

Observation: 
- có hơn 5000 bộ phim, nhưng chỉ có 2400 đạo diễn, 2000 diễn viên chính, 3000 dv nhì: cạnh tranh thật khốc liệt ^0^
- đoán: 47 thứ ngôn ngữ nhưng tiếng Anh là chủ đạo, 65 quốc gia nhưng USA là chủ yếu. 
- mấy cột facebook_likes ko biết là lấy số like trước hay sau khi phim ra mắt. Vì sau khi phim ra mắt chắc chắn các profile đều có tăng likes, nên c nghĩ là các thông số này không solid lắm. 
- khi quan sát imdb_score cần chú ý tới num_vote_user và num_user_for_review, vì nhiều fim điểm cao bởi vì có vài chục người vote sẽ ko khách quan bằng phim điểm cao có mười mấy ngàn người vote. 

In [39]:
#these lines below are just for satisfying my curiosity ^^! Hope it help
#1
data['language'].unique()

array(['English', nan, 'Japanese', 'French', 'Mandarin', 'Aboriginal',
       'Spanish', 'Filipino', 'Hindi', 'Russian', 'Maya', 'Kazakh',
       'Telugu', 'Cantonese', 'Icelandic', 'German', 'Aramaic', 'Italian',
       'Dutch', 'Dari', 'Hebrew', 'Chinese', 'Mongolian', 'Swedish',
       'Korean', 'Thai', 'Polish', 'Bosnian', 'None', 'Hungarian',
       'Portuguese', 'Danish', 'Arabic', 'Norwegian', 'Czech', 'Kannada',
       'Zulu', 'Panjabi', 'Tamil', 'Dzongkha', 'Vietnamese', 'Indonesian',
       'Urdu', 'Romanian', 'Persian', 'Slovenian', 'Greek', 'Swahili'],
      dtype=object)

In [31]:
#2
data['facenumber_in_poster'].unique()

array([ 0.,  1.,  4.,  3.,  2.,  6.,  7.,  5.,  8., nan, 10., 15.,  9.,
       11., 12., 31., 14., 19., 13., 43.])

In [32]:
#3
data['imdb_score'].unique()

array([7.9, 7.1, 6.8, 8.5, 6.6, 6.2, 7.8, 7.5, 6.9, 6.1, 6.7, 7.3, 6.5,
       7.2, 8.1, 7. , 7.7, 8.2, 5.9, 6. , 5.7, 6.4, 6.3, 5.6, 8.3, 8. ,
       8.4, 5.8, 5.4, 9. , 4.8, 5.2, 7.6, 4.5, 5.5, 8.6, 8.8, 5.1, 7.4,
       4.2, 5. , 4.9, 3.7, 5.3, 4.3, 3.8, 4.4, 3.3, 2.2, 8.9, 8.7, 4.6,
       2.4, 3.4, 4.1, 4.7, 3. , 3.6, 3.5, 2.7, 1.7, 4. , 2. , 9.3, 2.9,
       3.9, 2.8, 2.3, 1.9, 3.1, 9.5, 9.1, 1.6, 2.5, 2.1, 3.2, 9.2, 2.6])

In [33]:
#4 
data['content_rating'].unique()

array(['PG-13', nan, 'PG', 'G', 'R', 'TV-14', 'TV-PG', 'TV-MA', 'TV-G',
       'Not Rated', 'Unrated', 'Approved', 'TV-Y', 'NC-17', 'X', 'TV-Y7',
       'GP', 'Passed', 'M'], dtype=object)

Giải thích: 
- G – General Audiences - All ages admitted. Nothing that would offend parents for viewing by children.

- PG – Parental Guidance Suggested - Some material may not be suitable for children. Parents urged to give "parental guidance". May contain some material parents might not like for their young children.

- PG-13 – Parents Strongly Cautioned - Some material may be inappropriate for children under 13. Parents are urged to be cautious. Some material may be inappropriate for pre-teenagers.

- R – Restricted - Under 17 requires accompanying parent or adult guardian. Contains some adult material. Parents are urged to learn more about the film before taking their young children with them.

- NC-17 – Adults Only - No One 17 and Under Admitted. Clearly adult. Children are not admitted.
- TV-Y: appropriate for children of all ages. 
- TV-Y7:  designed for chil- dren ages 7 and older
- TV-14: unsuitable for children under the age of 14 
- TV-PG: 
- NC-17: No Children 17 and Under Admitted
- X : explicit films. Films rated X are intended only for viewing by adults, usually defined as people over the age of 18 or 21.
- GP:  All Ages Admitted – Parental Guidance Suggested 
- Unrated: In older movies, unrated means that the picture was made before the MPAA began using the rating system. In new movies, a DVD that says "unrated" means scenes have been added to the film that were not seen in the "rated" version
- Approved, Passed: old rating back in 1930 by MPPC code, mean the movie is approved to release/passed the jury review.

In [34]:
#5
data_sort=data.sort_values(by='imdb_score', ascending=False)
data_sort.head(3)


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
2765,Color,John Blanchard,NaN,65.0,0.0,176.0,Andrea Martin,770.0,NaN,Comedy,...,NaN,English,Canada,NaN,NaN,NaN,179.0,9.5,1.33,0
1937,Color,Frank Darabont,199.0,142.0,0.0,461.0,Jeffrey DeMunn,11000.0,28341469.0,Crime|Drama,...,4144.0,English,USA,R,25000000.0,1994.0,745.0,9.3,1.85,108000
3466,Color,Francis Ford Coppola,208.0,175.0,0.0,3000.0,Marlon Brando,14000.0,134821952.0,Crime|Drama,...,2238.0,English,USA,R,6000000.0,1972.0,10000.0,9.2,1.85,43000


In [35]:
#test code bỏ bớt cột để dễ nhìn hơn
drop_columns=['color','facenumber_in_poster','plot_keywords','movie_imdb_link','aspect_ratio','movie_facebook_likes','actor_2_facebook_likes','cast_total_facebook_likes','actor_1_facebook_likes','actor_3_facebook_likes','director_facebook_likes']
data_sort.drop(drop_columns,inplace=True, axis=1)
data_sort.head(3)

,director_name,num_critic_for_reviews,duration,actor_2_name,gross,genres,actor_1_name,movie_title,num_voted_users,actor_3_name,num_user_for_reviews,language,country,content_rating,budget,title_year,imdb_score
2765,John Blanchard,NaN,65.0,Andrea Martin,NaN,Comedy,Martin Short,Towering Inferno,10,Joe Flaherty,NaN,English,Canada,NaN,NaN,NaN,9.5
1937,Frank Darabont,199.0,142.0,Jeffrey DeMunn,28341469.0,Crime|Drama,Morgan Freeman,The Shawshank Redemption,1689764,Bob Gunton,4144.0,English,USA,R,25000000.0,1994.0,9.3
3466,Francis Ford Coppola,208.0,175.0,Marlon Brando,134821952.0,Crime|Drama,Al Pacino,The Godfather,1155770,Robert Duvall,2238.0,English,USA,R,6000000.0,1972.0,9.2


## 2. Adding metrics

Pick a side: pratical investor

What is ROI? https://corporatefinanceinstitute.com/resources/knowledge/finance/return-on-investment-roi-formula/ 

ROI Formula: <chèn ảnh vào cũng đc nhé, cho nó phong phú bài làm>


Add ROI column to dataframe, calculate ROI of each movie

## 3. Processing data

3.1 Cleaning data

- Remove all rows that has n/a ROI. 

- Verify dummies data (hàm get_dummies) -- remove rows if it affects to the ROI value

- Round up float64 number to interger ???

- Sort this new dataframe in descending order--> Take top_100_success as the new dataframe to work with. 

- Also take top_100_failures dataframe, - top_100_imdb dataframe for the Extra part to verify our hypotheses. 

!!! Chú ý khi làm data top_100_imdb: chỉ lấy những fim có lượng vote lớn do lượng vote nhỏ quá sẽ không được khách quan. 

3.2 Processing data

- tách genre ra
- phân thành group để khi visualize lên có các màu khác nhau.
- other ideas?


## 4. Visualization & observation

- Plot this "top100" in different angles to see if any interesting thing appear.
- Try to do simple charts to have a better understanding for audience. ( Last time team chị làm được mấy cái buble chart đẹp quá trời đẹp nhưng bị comment là hơi phức tạp - nếu present cho vd như CEO chẳng hạn thì vô nghĩa. Anw trong lớp thì được phép làm thoải mái, nhưng khi present phải dành time để giải thích chart kĩ càng cho mn hiểu. )

- Maybe we even need to dig down more information that aren't in the dataframe to see a pattern. Lets discuss this on saturday

## 5. Extra


Verify if the same observation of top_100_success happen in top_100_failures & top_100_imdb or not

# 6. Conclusion